### This is the first experiment where we will be implementing the RNN

In [195]:
import pandas as pd
import numpy as np

In [215]:
xls = pd.ExcelFile('PhyAdRsnComorbMed.xlsx')
df1 = pd.read_excel(xls, 'Hospital-length-of-stay')
df2 = pd.read_excel(xls, 'Days-breakdown')

In [216]:
df1 = df1[df1['did_the_patient_expire_in_hospital']=='No'][['parent_id','hospital_length_of_stay']]
df1

,parent_id,hospital_length_of_stay
1,2,5
2,3,7
5,6,32
6,7,4
7,8,10
...,...,...
502,516,13
504,519,9
505,521,7
506,522,5


In [217]:
df = df2.merge(df1, how='inner', on='parent_id')
df.head(20)

,id,parent_id,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,...,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,hospital_length_of_stay
0,9,2,1,122.0,80.0,72.0,24.0,92.0,36.3,NaN,...,No,"""[\""Bilateral Ground Glass\"",\""Bilateral Conso...",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
1,10,2,2,114.0,72.0,60.0,18.0,92.0,35.9,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
2,11,2,3,117.0,73.0,62.0,16.0,95.0,36.0,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
3,12,2,4,125.0,76.0,76.0,18.0,97.0,36.3,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
4,13,2,5,133.0,85.0,73.0,20.0,94.0,36.7,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
5,14,2,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
6,15,2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
7,16,2,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
8,17,3,1,112.0,69.0,66.0,18.0,96.0,36.5,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,Yes,Yes,7
9,18,3,2,150.0,93.0,67.0,19.0,93.0,36.1,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,Yes,Yes,7


### Handling null values

### Initially we will handle the null values for the columns which cant have null as the value for their respctive columns systolic_blood_pressure	,diastolic_blood_pressure, heart_rate, respiratory_rate, oxygen_saturation	and temperature
We will enter the mean for their respective ids

In [218]:
def enter_mean_values(df,column_name):
    for index,value in df[['parent_id',column_name]].iterrows():
       if pd.isna(value[1]):
            mean = df[df['parent_id']==value[0]][column_name].mean()
            df.loc[index, column_name] = mean

In [219]:
# List of columns you want to apply the function to
apply_mean_value = ['systolic_blood_pressure', 'diastolic_blood_pressure',
                    'heart_rate', 'respiratory_rate', 'oxygen_saturation', 'temperature',
                   'highest_heart_rate','lowest_heart_rate','highest_gcs','lowest_gcs']

for columns in apply_mean_value:
    enter_mean_values(df,columns)

In [222]:
df.head(50)

,id,parent_id,day,systolic_blood_pressure,diastolic_blood_pressure,heart_rate,respiratory_rate,oxygen_saturation,temperature,highest_mean_arterial_pressure,...,chest_x_ray,cxr_findings,chest_ct,chest_ct_findings,head_ct,head_ct_findings,antimicrobial,anticoagulation,steroid,hospital_length_of_stay
0,9,2,1,122.000000,80.000000,72.000000,24.000000,92.000000,36.300000,NaN,...,No,"""[\""Bilateral Ground Glass\"",\""Bilateral Conso...",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
1,10,2,2,114.000000,72.000000,60.000000,18.000000,92.000000,35.900000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
2,11,2,3,117.000000,73.000000,62.000000,16.000000,95.000000,36.000000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
3,12,2,4,125.000000,76.000000,76.000000,18.000000,97.000000,36.300000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",Yes,Yes,Yes,5
4,13,2,5,133.000000,85.000000,73.000000,20.000000,94.000000,36.700000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
5,14,2,6,122.200000,77.200000,68.600000,19.200000,94.000000,36.240000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
6,15,2,7,122.200000,77.200000,68.600000,19.200000,94.000000,36.240000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
7,16,2,14,122.200000,77.200000,68.600000,19.200000,94.000000,36.240000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,No,No,5
8,17,3,1,112.000000,69.000000,66.000000,18.000000,96.000000,36.500000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,Yes,Yes,7
9,18,3,2,150.000000,93.000000,67.000000,19.000000,93.000000,36.100000,NaN,...,No,"""[]""",No,"""[]""",No,"""[]""",No,Yes,Yes,7
